Este cuaderno no corresponde a una experimentación, si no a una visualización del modelo que se requeriría en este experimento y que a día de hoy SciANN no soporta.

In [1]:
%pip uninstall tensorflow-datasets tensorflow-metadata -y
%pip install tensorflow==2.9
%pip install sciann

Found existing installation: tensorflow-datasets 4.9.6
Uninstalling tensorflow-datasets-4.9.6:
  Successfully uninstalled tensorflow-datasets-4.9.6
Found existing installation: tensorflow-metadata 1.15.0
Uninstalling tensorflow-metadata-1.15.0:
  Successfully uninstalled tensorflow-metadata-1.15.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.7/511.7 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 58.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 57.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.7/438.7 kB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 57.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 85.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 47.9 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installa

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.6/169.6 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.4/561.4 kB 24.4 MB/s eta 0:00:00


In [2]:
import numpy as np
from sciann import Variable, Functional, SciModel
from sciann.constraints import Data
import sciann as sn
import matplotlib.pyplot as plt
from tensorflow import keras
import time

---------------------- SCIANN 0.7.0.1 ---------------------- 
For details, check out our review paper and the documentation at: 
 +  "https://www.sciencedirect.com/science/article/pii/S0045782520307374", 
 +  "https://arxiv.org/abs/2005.08803", 
 +  "https://www.sciann.com". 

 Need support or would like to contribute, please join sciann`s slack group: 
 +  "https://join.slack.com/t/sciann/shared_invite/zt-ne1f5jlx-k_dY8RGo3ZreDXwz0f~CeA" 
 
TensorFlow Version: 2.9.0 
Python Version: 3.10.12 (main, Mar 22 2024, 16:50:05) [GCC 11.4.0] 



Ejemplo simplificado para mostrar que actualmente SciANN no soporta la arquitectura requerida.

In [ ]:
#Número de sensores
m = 3

#Número de neuronas por capa
n_in_layer = 40

#Definición de las entradas
x = sn.Variable("x")
ux = [sn.Variable(f'ux{i}') for i in range(m)]


#Definición de subredes
subnet_branch = sn.Functional('branch_out', ux, [n_in_layer], activation='sigmoid')
subnet_trunk = sn.Functional("trunk_out",x ,[n_in_layer], 'sigmoid')

#Producto cartesiano de las salidas de ambas
s  = sn.utils.dot(subnet_branch,subnet_trunk)
d1 = sn.Data(s)

# Definimos el modelo
modelo_operador = sn.SciModel(inputs = ux+[x], targets=[d1], loss_func='MSE',optimizer="adam",plot_to_file="operador.png")

# Resumen del modelo
modelo_operador.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 ux0 (InputLayer)               [(None, 1)]          0           []                               
                                                                                                  
 ux1 (InputLayer)               [(None, 1)]          0           []                               
                                                                                                  
 ux2 (InputLayer)               [(None, 1)]          0           []                               
                                                                                                  
 concat_2 (Concatenate)         (None, 3)            0           ['ux0[0][0]',                    
                                                                  'ux1[0][0]',                

In [ ]:
#Número de sensores
m = 3

#Número de neuronas por capa
N = 5

#Definición de las entradas
x = sn.Variable("x")
ux = [sn.Variable(f'ux{i}') for i in range(m)]


#Definición de subredes

subnet_branches = [sn.Functional('branch_out_'+str(i), ux, [1], activation='sigmoid') for i in range(N)]
subnet_trunk = sn.Functional("trunk_out",x ,[N], 'sigmoid')

#Producto cartesiano de las salidas de ambas
probando = []
for i in range(N):
  probando.append(sn.utils.mul(subnet_branches[i],subnet_trunk))

s = sn.utils.add(probando[0],probando[1])

for i in range(2,N):
  s = sn.utils.add(s,probando[i])

d1 = sn.Data(s)

# Definimos el modelo
modelo_operador_2 = sn.SciModel(inputs = ux+[x], targets=[d1], loss_func='MSE',optimizer="adam",plot_to_file="operador_2.png")

# Resumen del modelo
modelo_operador.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 ux0 (InputLayer)               [(None, 1)]          0           []                               
                                                                                                  
 ux1 (InputLayer)               [(None, 1)]          0           []                               
                                                                                                  
 ux2 (InputLayer)               [(None, 1)]          0           []                               
                                                                                                  
 concat_2 (Concatenate)         (None, 3)            0           ['ux0[0][0]',                    
                                                                  'ux1[0][0]',                